In [1]:
import pandas as pd
from tqdm.notebook import tqdm_notebook
from random import randint
import re
from scipy import stats
import numpy as np

In [2]:
def parse_price(str):
    #converts price from ### ### грн. format to numeric format
    str_nocurrency = str.replace('грн', '')
    str_clean = str_nocurrency.replace(' ', '').replace('.', '')
    return int(str_clean)

def parse_tags(tags):
    tags_dict={}#'other_tags':[]}
    for tag in tags:
        if tag == 'Бізнес':
            tags_dict['posted_by']='Business'
        elif tag == 'Приватна особа':
            tags_dict['posted_by']='Private'
        elif tag[:12] == "Поверховість":
            tags_dict['building_height_floors']=float(tag.replace('Поверховість: ', '').replace(' ',''))
        elif tag[:6] == "Поверх":
            tags_dict['floor']=float(tag.replace('Поверх: ', '').replace(' ',''))
        elif tag[:8] == 'Загальна':
            tags_dict['apt_area_sqm']=float(tag.replace('Загальна площа: ', '').replace(' м²','').replace(' ',''))
        elif tag[:11] == 'Площа кухні':
            tags_dict['kitchen_area_sqm'] = float(tag.replace('Площа кухні: ', '').replace(' м²','').replace(' ',''))
        elif tag[:16] == 'Кількість кімнат':
            tags_dict['rooms'] = int(tag.replace('Кількість кімнат: ', '')[0])
        elif tag[:8] == 'Опалення':
            tags_dict['heating'] = tag.replace('Опалення: ', '')
        elif tag[:10] == 'Меблювання':
            tags_dict['furnishing'] = tag.replace('Меблювання: ', '')
        elif tag[:6] == 'Ремонт':
            tags_dict['renovation'] = tag.replace('Ремонт: ', '')
        elif tag[:17] == 'Домашні улюбленці':
            tags_dict['pets'] = tag.replace('Домашні улюбленці: ', '')
        elif tag[:10] == 'Планування':
            tags_dict['layout'] = tag.replace('Планування: ', '')
        elif tag[:11] == 'Тип будинку':
            tags_dict['building_type'] = tag.replace('Тип будинку: ', '')
        elif tag[:8] == 'Тип стін':
            tags_dict['building_material'] = tag.replace('Тип будинку: ', '')       
        elif tag[:8] == 'Cанвузол':
            tags_dict['bathroom'] = tag.replace('Cанвузол: ', '')
        #elif tag.lower() in ['договірна', 'поскаржитися', 'користувач', 'місцезнаходження', 'зараз онлайн', 'без комісії', 'готовий співпрацювати з ріелторами']:
        #    pass
        #else:
        #    tags_dict['other_tags'].append(tag)
    return tags_dict  

In [3]:
data_raw = pd.read_parquet ('Data/rent_details.parquet')

In [5]:
tags_df=pd.DataFrame(data_raw.tags.map(parse_tags).values.tolist())

In [6]:
data_parsed = data_raw.join(tags_df)
data_parsed.drop('tags', axis='columns', inplace=True)
data_parsed.price = data_parsed.price.map(parse_price)

In [7]:
data_parsed.isna().sum()

id                            0
url                           0
price                         0
negotiable                19257
description                   0
city                          0
date                          0
details                       0
district                      0
posted_by                     0
building_type              9859
floor                         1
building_height_floors        2
apt_area_sqm                  1
kitchen_area_sqm              2
building_material          9746
pets                      14181
rooms                         0
layout                     7791
bathroom                   6432
heating                    6404
renovation                 5620
furnishing                 4048
dtype: int64

In [8]:
data_parsed.dropna(subset=['apt_area_sqm'], inplace=True)

In [9]:
data_parsed.drop_duplicates(['price', 'description'], inplace=True)

In [10]:
cities=['Kiev', 'Lvov', 'Odessa', 'Kharkov', 'Dnepr']
# max floors, based on wiki article
max_floors = {'Kiev': 48, 'Dnepr': 28, 'Odessa': 25, 'Kharkov': 28, 'Lvov': 27}
# creating median
median_building_floors = data_parsed.groupby('city')['building_height_floors'].agg(pd.Series.median).to_dict()
median_apt_floor = data_parsed.groupby('city')['floor'].agg(pd.Series.median).to_dict()

In [11]:
for city in cities:
    data_parsed.loc[(data_parsed.city==city) & (data_parsed.floor > max_floors.get(city)), 'floor'] = median_apt_floor.get(city)
    data_parsed.loc[(data_parsed.city==city) & (data_parsed.building_height_floors > max_floors.get(city)), 'building_height_floors'] = median_building_floors.get(city)

In [12]:
data_parsed.drop(data_parsed[data_parsed.apt_area_sqm < data_parsed.rooms*12].index, inplace=True)

In [13]:
data_parsed['negotiable'] = data_parsed.negotiable.apply(lambda x: True if x == 'Договірна' else False)

In [14]:
data_parsed['lux'] = data_parsed.description.apply(lambda x: True if re.search('пентхаус|vip|вип', x.lower()) else False)

In [15]:
subset_large_kitchen = data_parsed.kitchen_area_sqm >= data_parsed.apt_area_sqm / 2
data_parsed.loc[subset_large_kitchen, 'kitchen_area_sqm'] = data_parsed[subset_large_kitchen].kitchen_area_sqm / 10

In [16]:
for city in cities:
    for room in range(1,6):
        subset = data_parsed.loc[(data_parsed.city == city) & (data_parsed.rooms == room)].index
        data_parsed.loc[subset, 'z_score'] = np.abs(stats.zscore(data_parsed.loc[subset].apt_area_sqm))



In [17]:
data_parsed = data_parsed.loc[data_parsed.z_score<3]

In [18]:
data_parsed.drop('z_score', axis=1, inplace=True)

In [19]:
data_parsed.to_csv('Data/rent_offers_clean.csv', index=False)